## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x112b1ba60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x112b51c90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Dhruv Agarwal and I am a Btech 3rd Year Student Studying the GenAi and currently studying the langchain framework")])

AIMessage(content="That's great, Dhruv!\n\nIt's awesome that you're diving into the world of Generative AI and exploring LangChain. It's a powerful framework for building applications with large language models. \n\nWhat specific aspects of LangChain are you focusing on in your studies?  Are there any particular projects or use cases that you're interested in?\n\nI'd love to hear more about your journey and maybe offer some insights or suggestions based on your interests.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 40, 'total_tokens': 145, 'completion_time': 0.190909091, 'prompt_time': 0.00156054, 'queue_time': 0.250791729, 'total_time': 0.192469631}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9d23bf70-1614-4d67-bfa1-9f13e4f16679-0', usage_metadata={'input_tokens': 40, 'output_tokens': 105, 'total_tokens': 145})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Dhruv Agarwal and I am a Btech 3rd Year Student Studying the GenAi and currently studying the langchain framework"),
        AIMessage(content="Hi Dhruv Agarwal,\n\nIt's great to meet you! \n\nIt's awesome that you're a 3rd-year BTech student interested in GenAI and LangChain. Those are exciting fields with a lot of potential. \n\nWhat specifically about LangChain are you working on right now? Are there any particular projects you're excited about?\n\nI'm happy to chat about GenAI, LangChain, or anything else related to AI that you'd like to discuss.  \n\nLet me know how I can help!\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Dhruv Agarwal, a 3rd-year BTech student studying GenAI and currently working with the LangChain framework.  \n\nIs there anything else you'd like to know or discuss? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 178, 'total_tokens': 226, 'completion_time': 0.087272727, 'prompt_time': 0.003986838, 'queue_time': 0.24717184, 'total_time': 0.091259565}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--dca526c1-8d9b-4b71-bd95-94893f68b88a-0', usage_metadata={'input_tokens': 178, 'output_tokens': 48, 'total_tokens': 226})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Dhruv Agarwal and I am a Btech 3rd Year Student Studying the GenAi and currently studying the langchain framework")],
    config=config
)

In [9]:
response.content

"Hi Dhruv,\n\nIt's great to meet you!  GenAI and LangChain are both really exciting fields right now.  \n\nBeing a 3rd year BTech student studying GenAI, you're at a fantastic point to dive deep into these technologies.  \n\nWhat aspects of LangChain are you finding most interesting so far?  \n\nAre you working on any projects or experiments using it?\n\nI'd love to hear more about your journey with GenAI and LangChain!  Perhaps I can offer some insights or resources based on your interests.\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)


AIMessage(content='Your name is Dhruv Agarwal.  😊 \n\nI remembered it from our earlier conversation!  \n\n\n\nIs there anything else I can help you with today, Dhruv?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 174, 'total_tokens': 214, 'completion_time': 0.072727273, 'prompt_time': 0.003899769, 'queue_time': 0.251410839, 'total_time': 0.076627042}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c3571fee-3020-4b8e-9fcb-901fb1fb2c62-0', usage_metadata={'input_tokens': 174, 'output_tokens': 40, 'total_tokens': 214})

In [11]:
## change the config-->session id #Maintaining the another session for the chat history 
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Dhruv Agarwal.")],
    config=config1
)
response.content

"Hi Dhruv Agarwal, it's nice to meet you!  Is there anything I can help you with today? 😊 \n\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Dhruv Agarwal.  \n\nI remember! 😄  How can I help you today, Dhruv? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Dhruv")]})

AIMessage(content="Hi Dhruv, nice to meet you!  \n\nI'm happy to help with any questions you have. Ask away! 😊  What can I do for you today? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 32, 'total_tokens': 73, 'completion_time': 0.074545455, 'prompt_time': 0.00149012, 'queue_time': 0.255272301, 'total_time': 0.076035575}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--10318d44-41f4-4bf3-9672-0f0da1832111-0', usage_metadata={'input_tokens': 32, 'output_tokens': 41, 'total_tokens': 73})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Dhruv")],
    config=config
)

response

AIMessage(content="Hello Dhruv! It's nice to meet you. 👋 \n\nHow can I help you today?  😄 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 32, 'total_tokens': 62, 'completion_time': 0.054545455, 'prompt_time': 0.001483069, 'queue_time': 0.248354562, 'total_time': 0.056028524}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ab073ee0-f279-4018-b534-fde81f61ed4f-0', usage_metadata={'input_tokens': 32, 'output_tokens': 30, 'total_tokens': 62})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Dhruv. 😊  I remember!  \n\nIs there anything else I can help you with?\n'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Dhruv Agarwal")],"language":"Hindi"})
response.content

'नमस्ते ध्रुव अग्रवाल! 👋\n\nमुझे खुशी है कि आप मुझसे बात कर रहे हैं। आप क्या जानना चाहते हैं? मैं अपनी पूरी कोशिश करूँगा कि आपकी सहायता कर सकूं। 😊\n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Dhruv Agarwal")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते ध्रुव आगारवाल! 👋  मुझे आपकी मदद करने में खुशी हो रही है।  आप किस बारे में जानना चाहते हैं? 🤔 \n'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [24]:
response.content

'आपका नाम ध्रुव आगारवाल है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [31]:
import warnings
warnings.filterwarnings('ignore')

from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You said you like vanilla ice cream!  🍨\n'

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "What is 2 + 2?" 😊 \n\n\n\n'

In [28]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob! 😊  I remember.'

In [30]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked me "whats 2 + 2"  \n\n\n\n'